In [1]:
import sys
from datetime import datetime

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [2]:
original_write = sys.stdout.write

def my_write(string_text):
    if string_text == '\n':
        return # так кажется некравсиво решать второй столбец
    current_time = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    value = repr(current_time + string_text)
    original_write(value)

sys.stdout.write = my_write
print('1, 2, 3')
sys.stdout.write = original_write


'[2023-12-14 20:33:59]1, 2, 3'

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [3]:
import sys
from datetime import datetime


def timed_output(function):
    def wrapper(*args,**kwargs):
        original_write = sys.stdout.write
        def my_write(string_text):
            if string_text == '\n':
                return #опять так же
            current_time= datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            original_write(current_time + string_text)
        
        sys.stdout.write = my_write
        result = function(*args,**kwargs)
        sys.stdout.write = original_write
        return result
    return wrapper


@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')


print_greeting("Nikita")

[2023-12-14 20:33:59]Hello, Nikita!

Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [4]:
import sys


def redirect_output(file_path):
    def decorator(func):
        def wrapper(*args, **kwargs):
            original_stdout = sys.stdout
            try:
                with open(file_path, 'a+') as file:
                    sys.stdout = file
                    result = func(*args, **kwargs)
            finally:
                sys.stdout = original_stdout

            return result
        return wrapper
    return decorator


@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()


calculate()

In [6]:
with open('function_output.txt', 'r') as file:
    for row in file:
        print(row.strip())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321
